In [27]:
import numpy as np
import time
from pytimedinput import timedInput

In [24]:
!pip install pytimedinput

In [28]:
player_action = timedInput(timeout=2, resetOnInput=False, endCharacters='\n')

RuntimeError: timedInput() requires an interactive shell, cannot continue.

In [21]:
# Trade itself [x]
# Remove Stale [x]
# Human Interface
    # varying times for quotes

MIN_MEAN = 10
MAX_MEAN = 100
VARIANCE = 12
CROSS_PROB = 0.4

Bid = 1
Offer = 0
Buy = 1
Sell = -1
side_map = {1: "Bid", 0: "Offer"}

class Quote:
    def __init__(self, price, side) -> None:
        self.price = round(price)
        self.side = side  # 1 = bid, # 0 = offer

    def __repr__(self):
        return f"{side_map[self.side]} {self.price}"

class Book:
    def __init__(self):
        self.bids = []
        self.offers = []
    
    def display(self):
        bids = sorted(self.bids, reverse=True)
        offers = sorted(self.offers, reverse=True)

        for o in offers:
            print(f'    | {o} ')
        for b in bids:
            print(f' {b} |  ')

    def clean_book(self):
        if len(self.offers) > 5:
            worst_offer = max(self.offers)
            self.offers.remove(worst_offer)
            print(f"Removed {worst_offer} Offer")
        if len(self.bids) > 5:
            worst_bid = min(self.bids)
            self.bids.remove(worst_bid)
            print(f"Removed {worst_bid} Bid")

    def append(self, quote):
        # Appends to order book (quote is not in cross)
        print(quote)
        if quote.side:
            self.bids.append(quote.price)
        else:
            self.offers.append(quote.price)

        self.clean_book()

    def process_quote(self, quote):
        # Quote is in cross
        best_offer = min(self.offers) if self.offers else 1e99
        best_bid = max(self.bids) if self.bids else 0
        lift = quote.side == Bid and quote.price > best_offer
        hit = quote.side == Offer and quote.price < best_bid

        if lift:
            print(f"{best_offer} Offer Lifted")
            self.offers.remove(best_offer)
        elif hit:
            print(f"{best_bid} Bid Hit")
            self.bids.remove(best_bid)
        else:
            self.append(quote)
       
class Bot:
    def __init__(self):
        self.theo = np.random.uniform(MIN_MEAN, MAX_MEAN)

    def quote(self):
        price = np.random.normal(self.theo, VARIANCE) 
        cross = np.random.random() < CROSS_PROB        
        side = int(price < self.theo and not cross)

        return Quote(price, side)


class Game:
    def __init__(self, iterations = 60):
        self.bot = Bot()
        self.book = Book()
        self.iterations = iterations
        self.position = 0
        self.settlement = self.bot.theo
        self.trades = []

    
    def start(self):
        for _ in range(self.iterations):
            q = self.bot.quote()
            self.book.process_quote(q)
            player_action, missed = timedInput(timeout=2, resetOnInput=False, endCharacters='\r')

            if not missed:
                self.execute_action(player_action)

            self.book.display()
            player_action = timedInput(timeout=3, resetOnInput=False, endCharacters='\r')

            if not missed:
                self.process_action(player_action)

            print("-"*80)

    def process_action(self, action):
        """Action must be hit, lift, mine or yours"""
        best_bid = max(self.book.bids)
        best_offer = min(self.book.offers)

        if action in ['hit', 'yours']:
            self.execute(Sell, best_offer)

        if action in ['lift', 'mine']:
            self.execute(Buy, best_bid)

    
    def execute(self, side, price):
        best_offer = best_bid = price
        self.position += side
        self.trades.append(side * price)
        if side > 0:
            print(f"Bought @ {best_offer}!")
            self.book.offers.remove(best_offer)
        else:
            print(f"Sold @ {best_bid}!")
            self.book.bids.remove(best_bid)


In [23]:
g.bot.theo

16.226140108184836

In [22]:
g = Game()
g.start()

Offer 26
    | 26 
--------------------------------------------------------------------------------
Offer 29
    | 29 
    | 26 
--------------------------------------------------------------------------------
Bid 9
    | 29 
    | 26 
 9 |  
--------------------------------------------------------------------------------
Offer 36
    | 36 
    | 29 
    | 26 
 9 |  
--------------------------------------------------------------------------------
Offer 14
    | 36 
    | 29 
    | 26 
    | 14 
 9 |  
--------------------------------------------------------------------------------
Offer 29
    | 36 
    | 29 
    | 29 
    | 26 
    | 14 
 9 |  
--------------------------------------------------------------------------------
Bid 12
    | 36 
    | 29 
    | 29 
    | 26 
    | 14 
 12 |  
 9 |  
--------------------------------------------------------------------------------
Offer 30
Removed 36 Offer
    | 30 
    | 29 
    | 29 
    | 26 
    | 14 
 12 |  
 9 |  
---------------------

KeyboardInterrupt: 

In [ ]:
g.bo

In [215]:
bk = Book()
bk.bids = [21, 25, 29, 31]
bk.offers = [34, 35, 37]

bk.display()

    | 34 
    | 35 
    | 37 
 21 |  
 25 |  
 29 |  
 31 |  


In [236]:
b = Bot()

In [217]:
b.theo

43.53391091461828

In [245]:
q.side

0

In [243]:
q = b.quote()
print(q)

Offer 41


In [114]:
price = np.random.normal(100, VARIANCE)
print(price)
cross = np.random.random() < CROSS_PROB 
print(cross)       
side = int(price < 100 and not cross)
print(side)

142.73100432074634
True
0
